In [22]:
import matplotlib
import numpy as np
import math
import itertools
import sklearn
import pydotplus
import warnings
import mysql.connector as myconnect
import pandas as pd
import sklearn.linear_model
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import tree
from scipy import stats
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from prompred import *
from sklearn.preprocessing import normalize
from sklearn import preprocessing
%matplotlib qt

In [16]:
dash = np.empty([35],dtype=np.dtype(str,1))
dash.fill('a')
dash[:3]='b'


dash

array(['b', 'b', 'b', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a',
       'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a', 'a'], 
      dtype='<U1')

In [7]:
#   spacer length
#   frequency AGCT
#   position base
#   %AT / %GC
#   difference 

#   binding affinity

#dataset vergroten adhv bootstrap
#Histogram 
#Principal component analysis -> kleurenmap adhv expressiematrix -> kleurenmap hoog contrast
#normalize ridge/lasso
#support vector machines -> kernel voor sequenties.

"""
Best Feature Selection Method?
Alpha voor CV
Optimal alpha
"""

### Check voor identieke sequenties
### Optimalisatie
### Spearman Rank Coefficient
### Kendel Tau
### Covariantie

#variantie binnenin dataset
# gaussian processer
# Promoter data Composability of regulatory sequences controlling transcription and translation in Escherichia coli

# 4de machts wortel, kwadraat, dat delen door resultaat


'\nBest Feature Selection Method?\nAlpha voor CV\nOptimal alpha\n'

In [17]:
# Query sequences and fluorescence level
plt.close('all')

## RAND + MODULATED
dfDatasetOrder = querySQL('mutalik_prom_lib','(SELECT ID, sequence, mean_score, mean_score_sd, 35boxstart, 10boxstart FROM mod_prom_lib) UNION (SELECT ID, sequence, mean_score, mean_score_sd, 35boxstart, 10boxstart FROM rand_prom_lib);')

## RAND
##dfDatasetOrder = querySQL('mutalik_prom_lib','SELECT ID, sequence, mean_fluo, mean_fluo_sd, 35boxstart, 10boxstart  FROM rand_prom_lib')
## MODULATED
##dfDatasetOrder = querySQL('mutalik_prom_lib','SELECT ID, sequence, mean_score, mean_score_sd, 35boxstart, 10boxstart FROM mod_prom_lib')

dfDataset = dfDatasetOrder.reindex(np.random.permutation(dfDatasetOrder.index))

dataset = dfDataset.values
#print(dfDataset)

sequences= dataset[:,0]
yData = dataset[:,1]
start35Box = dataset[:,2]
start10Box = dataset[:,3]
ID = dataset[:,4]
mean_fluo_sd = dataset[:,5]
#


# Selecting regions


yData, positionBox, spacer = regionSelect(dfDataset, [[0,12],[-8,12]], [-35,1])

positionMatrix = positionBox.values
# Plot linear regression


In [18]:
np.size(labely)

254

In [23]:
plt.close('all')

#----parEvalS --------------
Xdf = positionBox
#X = preprocessing.scale(positionMatrix, axis=1)
X = positionMatrix
y = [math.sqrt(math.sqrt(u)) for u in yData]

regType = 'SVC'
poly= 3

kernel = 'poly'
parLabel = ['gamma']
parRange = [2] 

testSize = 0.20 
k = 2
kInner = 5
pdf = None


##OPTIONAL

treeCount = 100
gamma = 0.1
alpha = 0.1
coef0 = 0.00
#---------------------------
penalty = 0.1
epsilon = 1.95

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "treeCount":treeCount, "coef0":coef0}


#

if len(parLabel) is 1: 
    scoresParCV, optimalParCV = KfoldCV(X,y,k,parModel,parLabel[0],parRange[0]) 
    scoresParNCV, optimalParNCV, scoresNCV = nestedKfoldCV(X,y,k,kInner,parModel,parLabel[0],parRange[0])    
    meanScores = np.mean(np.ndarray.max(scoresParCV,axis=1))
    print("K FOLD CV \n---------- \n\n Maximum Score: ",np.max(scoresParCV), "\n Mean optimal score: ", meanScores ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-meanScores),2)))   , "\n Optimal parEval:\n", optimalParCV, "\n parEval Scores:\n", scoresParCV,"\n\n\n")
    print("NESTED K FOLD CV \n----------------- \n\n Maximum Score: ",np.max(scoresParNCV), "\n Mean optimal score: ", np.mean(scoresParNCV) ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-np.mean(scoresParNCV)),2))) , "\n Optimal parEval:\n", optimalParNCV, "\n parEval Scores:\n", scoresParNCV,"\n\n\n")

if len(parLabel) is 2:
    evaluateMultiPar(X, y, parModel, parLabel, parRange)
    
    
if (regType in ["ridge","lasso","OLS"] and poly is None): 
    reg = selectRegression(regType, alpha=np.median(optimalParCV), poly=poly, kernel=kernel) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize) 
    reg.fit(X_train,y_train) 
    plotPositionCoefficients(reg, positionBox, posRange)


    

ValueError: Unknown label type: 'continuous'

In [42]:
## Show Out

XnoOut = positionMatrix[np.invert(OutI),:]
YnoOut = np.extract(np.invert(OutI),y)

IDout, seqOut, scoreOut = np.extract(OutI,ID),np.extract(OutI,sequences),np.extract(OutI, OutV)
fluoOut, fluoSdOut, spacerOut = np.extract(OutI,yData), np.extract(OutI, mean_fluo_sd),np.extract(OutI,spacer)
difLength = np.max(np.extract(OutI,start10Box))-np.extract(OutI,start10Box)
sequenceAligned = ["-" *difLength[u]+seqOut[u] for u in range(np.shape(seqOut)[0])]
np.transpose(np.vstack((IDout, sequenceAligned, scoreOut, fluoOut, fluoSdOut, spacerOut)))

#[[sequences[u], OutV[u]] for u, i in enumerate(OutI) if i==False]

NameError: name 'OutI' is not defined

In [43]:
plt.close('all')

#----parEvalS --------------
Xdf = positionBox
#X = XnoOut
#y = YnoOut

regType = 'ridge'
poly= 3

kernel = 'poly'
parLabel = ['alpha']
parRange = [15] 

testSize = 0.20 
k = 5 
kInner = 5
pdf = None


##OPTIONAL

treeCount = 100
gamma = 0.1
alpha = 0.1
coef0 = 0.01
#---------------------------
penalty = 0.1
epsilon = 1.95

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "treeCount":treeCount, "gamma":gamma, "coef0":coef0}

test = sklearn.svm.SVR( C=penalty, kernel=kernel, gamma=gamma, coef0=coef0, epsilon=epsilon, degree=poly)
test.fit(X,y)
#

if len(parLabel) is 1: 
    scoresParCV, optimalParCV, OutI, OutV = KfoldCV(X,y,k,parModel,parLabel[0],parRange[0]) 
    scoresParNCV, optimalParNCV, scoresNCV = nestedKfoldCV(X,y,k,kInner,parModel,parLabel[0],parRange[0])    
    meanScores = np.mean(np.ndarray.max(scoresParCV,axis=1))
    print("K FOLD CV \n---------- \n\n Maximum Score: ",np.max(scoresParCV), "\n Mean optimal score: ", meanScores ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-meanScores),2)))   , "\n Optimal parEval:\n", optimalParCV, "\n parEval Scores:\n", scoresParCV,"\n\n\n")
    print("NESTED K FOLD CV \n----------------- \n\n Maximum Score: ",np.max(scoresParNCV), "\n Mean optimal score: ", np.mean(scoresParNCV) ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-np.mean(scoresParNCV)),2))) , "\n Optimal parEval:\n", optimalParNCV, "\n parEval Scores:\n", scoresParNCV,"\n\n\n")

if len(parLabel) is 2:
    evaluateMultiPar(X, y, parModel, parLabel, parRange)
    
if (regType in ["ridge","lasso","OLS"] and poly is None): 
    reg = selectRegression(regType, alpha=np.median(optimalParCV), poly=poly, kernel=kernel) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize) 
    reg.fit(X_train,y_train) 
    plotPositionCoefficients(reg, positionBox, posRange)


ValueError: not enough values to unpack (expected 4, got 2)

In [5]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM anderson_prom_lib')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-35,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[0,12],[-12,12]], posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[0,12],[-12,12]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

print(np.size(X))
print(np.size(Xtest))

#print(np.shape(dfDatasetTrain))
print(ytest)

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3

#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------
"""
parModel = {"regType":regType, "poly":poly, "kernel":kernel }

parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)"""

#---------------------------
#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":0.001, "alpha": 0.001, "coef0":0.001 } 
reg = selectRegression(**parModelTweak)
#reg = GS.best_estimator_
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')



38100
2850
[1.0 0.86 0.72 0.7 0.58 0.56 0.51 0.47 0.36 0.33 0.24 0.16 0.15 0.1 0.06
 0.04 0.01 0.01 0.0]
[['TTGACGGCTAGCTCAGTCCTAGGTACAGTGCTAGC' 1.0 0.9592859060253289]
 ['TTGACAGCTAGCTCAGTCCTAGGTACTGTGCTAGC' 0.86 0.9223489602844605]
 ['TTGACAGCTAGCTCAGTCCTAGGTATTGTGCTAGC' 0.72 1.0419421285064308]
 ['TTTACAGCTAGCTCAGTCCTAGGTATTATGCTAGC' 0.7 0.9860207058854684]
 ['TTGACGGCTAGCTCAGTCCTAGGTATAGTGCTAGC' 0.58 1.0864933433798916]
 ['TTGACGGCTAGCTCAGTCCTAGGTATTGTGCTAGC' 0.56 0.940685838637401]
 ['CTGACAGCTAGCTCAGTCCTAGGTATAATGCTAGC' 0.51 1.1366213708095692]
 ['TTTACGGCTAGCTCAGTCCTAGGTATAGTGCTAGC' 0.47 0.9048435579522479]
 ['TTTACGGCTAGCTCAGCCCTAGGTATTATGCTAGC' 0.36 0.9167794384497944]
 ['TTTACGGCTAGCTCAGTCCTAGGTACAATGCTAGC' 0.33 0.9057306125453372]
 ['TTTACGGCTAGCTCAGTCCTAGGTACTATGCTAGC' 0.24 0.7763212956297116]
 ['TTGACAGCTAGCTCAGTCCTAGGGACTATGCTAGC' 0.16 0.9181584583331549]
 ['TTTATAGCTAGCTCAGCCCTTGGTACAATGCTAGC' 0.15 0.8996674904673486]
 ['TTTATGGCTAGCTCAGTCCTAGGTACAATGCTAGC' 0.1 0.8098252

In [19]:

dfDatasetTest = querySQL('prom_test_data', 'SELECT  ID, sequence, mean_fluo, 35boxstart, 10boxstart FROM inbio_prom_lib')
dfDatasetTest.to_csv(path_or_buf="/home/jim/Doctoraat/Code/data/inbio_lib.csv", index=False)
dfDatasetTest = querySQL('prom_test_data', 'SELECT  ID, sequence, mean_fluo, 35boxstart, 10boxstart FROM anderson_prom_lib')
dfDatasetTest.to_csv(path_or_buf="/home/jim/Doctoraat/Code/data/anderson_lib.csv", index=False)
dfDatasetTest = querySQL('prom_test_data', 'SELECT  ID, sequence, mean_fluo, 35boxstart, 10boxstart FROM brewster_prom_lib')
dfDatasetTest.to_csv(path_or_buf="/home/jim/Doctoraat/Code/data/brewster_lib.csv", index=False)
dfDatasetTest = querySQL('prom_test_data', 'SELECT ID ,sequence, mean_fluo, 35boxstart, 10boxstart FROM hammer_prom_lib')
dfDatasetTest.to_csv(path_or_buf="/home/jim/Doctoraat/Code/data/hammer_lib.csv", index=False)

In [26]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM brewster_prom_lib')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-40,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[-5,13],[-6,12]],posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[-5,13],[-6,12]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]


#print(np.array(positionBoxTrain.columns))
#print(np.array(positionBoxTest.columns))

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3


#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel }

"""parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)"""

#---------------------------

#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":0.0001, "alpha":0.1, "coef0":0.1 } 
####
reg = selectRegression(**parModelTweak)
#reg = GS.best_estimator_
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')



[['CAGGCTTTACACTTTATGCTTCCGGCTCGGTTGTAGTGTGG' 81.46 2.252786289810921]
 ['CAGGCCTCAGACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 76.0 2.2579654675406733]
 ['CAGGCCTTAGACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 69.93 2.258039776750028]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGGTTAAAGTGTGG' 69.81 2.262938208700772]
 ['CAGGCTTAAGACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 69.03 2.264208960552707]
 ['TCGAGCTCAGACTTTATGCTTCCGGCTCGTATAATGTGTGG' 64.35 2.268143731598088]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGGATAACGTGTGG' 62.58 2.2686194838553897]
 ['CAGGCATTACACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 57.83 2.263616397531156]
 ['TCGAGTTAAGACTTTATGCTTCCGGCTCGTATAATGTGTGG' 57.38 2.274405600470362]
 ['CAGGCTTTACACTTTATGCTTCCGGCTCGTATGTTGTGTGG' 53.45 2.270314841106836]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGGATAATGTGTGG' 51.76 2.2754154184724538]
 ['TCGTGTTTACCCTTTATGCTTCCGGCTCGTATAATGTGTGG' 49.57 2.276573606087575]
 ['TCGAGTTTACACTTTATGCTTCCGGCTCGAATAATGTGTGG' 46.74 2.2760239028840727]
 ['TCGAGATTACACTTTATGCTTCCGGCTCGTATAATGTGTGG' 46.18 2.2738050293983383]
 [

In [4]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM inbio_prom_lib WHERE length(sequence)>115')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-45,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[-7,12],[-8,11]],posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[-7,12],[-8,11]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3

#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel }

"""parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)"""

#---------------------------

#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":0.0001, "alpha": 10000, "coef0":0.1 } 
####
#reg= GS.best_estimator_
reg = selectRegression(**parModelTweak)
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
#print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')

SpearmanrResult(correlation=0.12627265162638146, pvalue=0.37238869055788715)


In [13]:
##################################
### TEST SETS
plt.close('all')
###################################


dfDatasetTrain = querySQL('mutalik_prom_lib','(SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM mod_prom_lib) UNION (SELECT sequence, mean_fluo, 35boxstart, 10boxstart, ID, mean_fluo_sd FROM rand_prom_lib);')
dfDatasetTrain = dfDatasetTrain.reindex(np.random.permutation(dfDatasetTrain.index))
datasetTrain = dfDatasetTrain.values

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM hammer_prom_lib')
dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
datasetTest = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


posRange = [-40,1]
positionBoxTrain, spacerTrain = regionSelect(datasetTrain, [[-7,14],[-9,12]], posRange)
positionBoxTest, spacerTest = regionSelect(datasetTest, [[-7,14],[-9,12]], posRange)

positionMatrixTrain = positionBoxTrain.values
positionMatrixTest = positionBoxTest.values


Xdf = positionBoxTrain
X = positionMatrixTrain
y = [math.sqrt(math.sqrt(u)) for u in datasetTrain[:,1]]

#X = XnoOut
#y = YnoOut
Xtest = positionMatrixTest
ytest = datasetTest[:,1]

#############################
regType = 'ridge'
kernel = 'poly'
poly= 3

#----parEvalS --------------

parLabel = ['gamma','alpha','coef0']
parRange = [10,10,10] 

#---------------------------

"""parModel = {"regType":regType, "poly":poly, "kernel":kernel }

parEval = getParameters(parLabel,parRange)
reg = selectRegression(**parModel)
GS = GridSearchCV(reg, parEval)
GS.fit(X,y)
print(GS.best_estimator_)
"""
#---------------------------

#########################
parModelTweak =  {"regType":regType, "poly":poly, "kernel":kernel, "gamma":10, "alpha":0.1, "coef0":0.1 } 
####
reg = selectRegression(**parModel)
reg.fit(X,y)
rankPredict = reg.predict(Xtest)
print(np.transpose(np.vstack((datasetTest[:,0],datasetTest[:,1],rankPredict))))

print(stats.spearmanr(datasetTest[:,1],rankPredict))
plt.plot(datasetTest[:,1],rankPredict, 'ro')





[['CTTTGGCAGTTTATTCTTGACATGTAGTGAGGGGGCTGGTATAATCACATAGTACTGTT' 528.0
  3.036887037555399]
 ['CATGTGGGAGTTTATTCTTGACACAGATATTTCCGGATGATATAATAACTGAGTACTGTT' 280.0
  2.716241258271957]
 ['TATGCGGTAGTTTATTCTTGACATGACGAGACAGGTGTGGTATAATGGGTCTAGATTAGG' 134.0
  3.0525260233182774]
 ['CATATACAAGTTTATTCTTGACACTAGTCGGCCAAAATGATATAATACCTGAGTACTGTT' 101.0
  2.1881650165053452]
 ['CATAGAGAAGTTTATTCTTGACAGCTAACTTGGCCTTTGATATAATACATGAGTACTGTT' 92.0
  2.3769709971684048]
 ['CATACGGGAGTTTATTCTTGACATATTGCCGGTGTGTTGGTATAATAACTTAGTACTGTT' 60.0
  3.6688988383684578]
 ['CATAGTCTAGTTTATTCTTGACACGCGGTCCATTGGCTGGTATAATAATTTAGTACTGTT' 56.0
  3.089457607879062]
 ['CATGACAGAGTTTATTCTTGACAGTATTGGGTTACTTTGGTATAATAGTTGAGTACTGTT' 42.0
  2.581752498639796]
 ['CATCGGGTAGTTTATTCTTGACAATTAAGTAGAGCCTGATATAATAGTTCAGTACTGTT' 34.0
  2.5285787706578793]
 ['CATGATGTAGTTTATTCTTGACACTGAGAGGGCCTCTTGATATAATAGTTGAGTACTGTT' 33.0
  2.5191920880142575]
 ['CATGGGTGAGTTTATTCTTGACAGTGCGGCCGGGGGCTGATATCATAGCAGAGTACTATT' 22.0
  2.32087564

In [32]:
plt.close('all')

#----PARAMETERS -----
Xdf = positionBox
X = XnoOut
y = YnoOut


regType = 'ridge'
poly= 3
gamma = 1
coef0 = 1

kernel = 'poly'
parLabel = ['alpha']
parRange = [15] 

testSize = 0.20 
k = 5 
kInner = 5
pdf = None
#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "gamma":gamma, "coef0":coef0}


if len(parLabel) is 1: 
    scoresParCV, optimalParCV, OutI, OutV = KfoldCV(X,y,k,parModel,parLabel[0],parRange[0]) 
    scoresParNCV, optimalParNCV, scoresNCV = nestedKfoldCV(X,y,k,kInner,parModel,parLabel[0],parRange[0])    
    meanScores = np.mean(np.ndarray.max(scoresParCV,axis=1))
    print("K FOLD CV \n---------- \n\n Maximum Score: ",np.max(scoresParCV), "\n Mean optimal score: ", meanScores ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-meanScores),2)))   , "\n Optimal parEval:\n", optimalParCV, "\n parEval Scores:\n", scoresParCV,"\n\n\n")
    print("NESTED K FOLD CV \n----------------- \n\n Maximum Score: ",np.max(scoresParNCV), "\n Mean optimal score: ", np.mean(scoresParNCV) ,"\n sd optimal scores: ", math.sqrt(np.sum(np.power((np.ndarray.max(scoresParCV,axis=1)-np.mean(scoresParNCV)),2))) , "\n Optimal parEval:\n", optimalParNCV, "\n parEval Scores:\n", scoresParNCV,"\n\n\n")
if len(parLabel) is 2:
    evaluateMultiPar(X, y, parModel, parLabel, parRange)
    
if (regType in ["ridge","lasso","OLS"] and poly is None): 
    reg = selectRegression(regType, alpha=np.median(optimalParCV), poly=poly, kernel=kernel) 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize) 
    reg.fit(X_train,y_train) 
    plotPositionCoefficients(reg, positionBox, posRange)


K FOLD CV 
---------- 

 Maximum Score:  0.8636674365 
 Mean optimal score:  0.82419440891 
 sd optimal scores:  0.056555113388595396 
 Optimal parEval:
 [  1000.  10000.   1000.   1000.  10000.] 
 parEval Scores:
 [[ 0.70339157  0.70339157  0.70339157  0.70339157  0.70339163  0.70339216  0.70339752  0.70345111  0.70398418
   0.70905269  0.74225203  0.78447419  0.71069898  0.42680514 -0.86323757]
 [ 0.77386117  0.77386117  0.77386117  0.77386117  0.7738612   0.77386151  0.77386461  0.77389557  0.77420363
   0.77713337  0.7966842   0.83691832  0.86366744  0.6795331  -0.44599599]
 [ 0.81285496  0.81285496  0.81285496  0.81285496  0.81285499  0.81285519  0.81285725  0.81287783  0.8130817   0.81494989
   0.8248565   0.83073847  0.79828143  0.54095783 -0.91545982]
 [ 0.81226191  0.81226191  0.81226191  0.81226192  0.81226193  0.81226204  0.81226322  0.81227492  0.81239128
   0.81348609  0.82016728  0.82095822  0.75534359  0.52605366 -0.71299687]
 [ 0.7616594   0.7616594   0.7616594   0.7616

In [11]:

dfDatasetTest = querySQL('prom_test_data', 'SELECT  sequence, mean_fluo, 35boxstart, 10boxstart, ID  FROM hammer_prom_lib')

dfDatasetTest['sequence'] = dfDatasetTest['sequence'].str.upper()
datasetUnsorted = dfDatasetTest.values
dataset = datasetUnsorted[datasetUnsorted[:,1].argsort()][::-1]


sequences= dataset[:,0]
yData = dataset[:,1]
start35Box = dataset[:,2]
start10Box = dataset[:,3]

ID = dataset[:,4]

posRange = [-35,1]
XtestBox, spacerTest = regionSelect(dataset, [[0,10],[-6,12]], test=positionBox)
Xtest = XtestBox.values

regType = 'ridge'
poly= 3
gamma = 1
coef0 = 1
alpha = 1000
kernel = 'poly'

testSize = 0.20 
k = 5 
kInner = 5
pdf = None
#---------------------------

parModel = {"regType":regType, "poly":poly, "kernel":kernel, "gamma":gamma, "coef0":coef0, "alpha":alpha}

reg = selectRegression(**parModel)
reg.fit(X,y)
rankPredict = reg.predict(Xtest)

print(np.transpose(np.vstack((sequences,yData,rankPredict))))

print(stats.spearmanr(yData,rankPredict))
print(stats.spearmanr(rankPredict,yData))



[['CTTTGGCAGTTTATTCTTGACATGTAGTGAGGGGGCTGGTATAATCACATAGTACTGTT' 528.0 5.708700659453884]
 ['CATGTGGGAGTTTATTCTTGACACAGATATTTCCGGATGATATAATAACTGAGTACTGTT' 280.0 5.708700659453884]
 ['TATGCGGTAGTTTATTCTTGACATGACGAGACAGGTGTGGTATAATGGGTCTAGATTAGG' 134.0 5.708700659453884]
 ['CATATACAAGTTTATTCTTGACACTAGTCGGCCAAAATGATATAATACCTGAGTACTGTT' 101.0 5.708700659453884]
 ['CATAGAGAAGTTTATTCTTGACAGCTAACTTGGCCTTTGATATAATACATGAGTACTGTT' 92.0 5.708700659453884]
 ['CATACGGGAGTTTATTCTTGACATATTGCCGGTGTGTTGGTATAATAACTTAGTACTGTT' 60.0 5.708700659453884]
 ['CATAGTCTAGTTTATTCTTGACACGCGGTCCATTGGCTGGTATAATAATTTAGTACTGTT' 56.0 5.708700659453884]
 ['CATGACAGAGTTTATTCTTGACAGTATTGGGTTACTTTGGTATAATAGTTGAGTACTGTT' 42.0 5.708700659453884]
 ['CATCGGGTAGTTTATTCTTGACAATTAAGTAGAGCCTGATATAATAGTTCAGTACTGTT' 34.0 5.399529980199178]
 ['CATGATGTAGTTTATTCTTGACACTGAGAGGGCCTCTTGATATAATAGTTGAGTACTGTT' 33.0 5.708700659453886]
 ['CATGGGTGAGTTTATTCTTGACAGTGCGGCCGGGGGCTGATATCATAGCAGAGTACTATT' 22.0 4.632584202374565]
 ['CATAGAACAGTTTATT

In [121]:
""" LINEAR REGRESSION USING OTHER ELEMENTS

# Query sequences, fluorescence level, reference regions

resultset =  np.squeeze(querySQL('mutalik_prom_lib','SELECT sequence, mean_fluo, 35boxstart, 10boxstart FROM rand_prom_lib').values)

sequences= resultset[:,0]
score = resultset[:,1]
start35Box = resultset[:,2]
start10Box = resultset[:,3]

# Selecting regions

box10 = selectRegions(sequences, [0,6], start10Box)
box35 = selectRegions(sequences, [0,6], start35Box)
spacer = start10Box-start35Box-6

# Creating features


fractionA_10, fractionT_10, fractionC_10, fractionG_10 = extractATCGpercentage(box10)
fractionA_35, fractionT_35, fractionC_35, fractionG_35 = extractATCGpercentage(box35)

fractionGC = [(o+p+q+r)/4 for o,p,q,r in zip(fractionC_10,fractionG_10,fractionC_35,fractionG_35)]
fractionAT = [(o+p+q+r)/4 for o,p,q,r in zip(fractionA_10,fractionT_10,fractionA_35,fractionT_35)]


featureMatrix = np.transpose(np.array([fractionGC,spacer]))


#print(featureMatrix)

print(np.shape(spacer),np.shape(score))
# Fit linear regression

linReg = LinearRegression(fit_intercept=True, normalize=True)
linReg.fit(featureMatrix, score)

#linReg.fit(np.squeeze(spacer), np.squeeze(score))

print(linReg.coef_)
print(linReg.intercept_)


# Plot coefficients
index = np.arange(np.shape(featureMatrix)[1])
coefPlot = plt.bar(index, linReg.coef_, 0.10 )
intPlot = plt.bar(-1, linReg.intercept_, 0.10)
plt.axhline(0, color='black')
plt.axvline(0, color='black')


(138,) (138,)
[-311.19084743   68.32928228]
-1000.49146613


In [ ]:
######################### EXPERIMENTAL GROUNDS ###############################################


"""
topScores= np.empty([8,8])
topCombinations = np.empty([8,8])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20)
logIt = [math.pow(10,u) for u in range(-3,5)]
for i in logIt:
    index = int(math.log10(i)+3)
    combinations, results = bestSubsetSelection(X_train, X_test, y_train, y_test,2, 'ridge', alpha=i)
    print(results[0:8])
    print(combinations[0:8])"""








